#### Haikuus are usually short poems consisting of three lines.Traditionaly the 1st line is 5 sylable the second is 7 and the third is 5

In [1]:
import pandas as pd
import numpy as np

import re

data = pd.read_csv("haikus.csv",header=None,encoding = "%%latin1")
data = data.rename({0:"Haikuu"},axis=1)
#data=data.iloc[0:100,:]
df = data.values
df

array([['rectory roofers\ntheir ladders\ntake them higher\n'],
       ['summer cabin\nthe ants\ndo the dishes\n'],
       ['lagoon at sunrise?\nthe shadow\nchases its pelican\n'],
       ...,
       ['spruce woods\nfireweed filling\nthe vacancy\n'],
       ['corrugated sun?\nchilies and laundry\nin rooftop haze\n'],
       ['home from war\nwe ease out\nthe champagne corks\n']],
      dtype=object)

#### Preprocessing the data(Dividing the Haikuu into 3 Lines(Original Structure!))

In [2]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)
pattern = '\\'
count = 0
for i in df:
    col1=[]
    col2=[]
    col3=[]
    pattern1 = ''
    positions = []
    my_lst_str=''
    my_lst_str1=''
    my_lst_str2=''
    pattern1 = str(i).strip('[]')
    positions = list(find_all(pattern1,pattern))
    if (len(positions) == 3):
        
        for j in range(positions[0]):
            col1.append(pattern1[j])
            my_lst_str = ''.join(map(str, col1)) 
    
        for j in range(positions[0]+2,positions[1]):
            col2.append(pattern1[j])
            my_lst_str1 = ''.join(map(str, col2))
            
        
        for j in range(positions[1]+2,positions[2]):
            col3.append(pattern1[j])
            my_lst_str2 = ''.join(map(str, col3)) 
    elif(len(positions) == 2):
        for j in range(positions[0]):
            col1.append(pattern1[j])
            my_lst_str = ''.join(map(str, col1)) 
    
        for j in range(positions[0]+2,positions[1]):
            col2.append(pattern1[j])
            my_lst_str1 = ''.join(map(str, col2))
        my_lst_str2 = '...'
    else:
        for j in range(positions[0]):
            col1.append(pattern1[j])
            my_lst_str = ''.join(map(str, col1)) 
        my_lst_str1 = '...'
        my_lst_str2 = '...'
    data.loc[count, '1st Verse'] = my_lst_str
    data.loc[count, '2nd Verse'] = my_lst_str1
    data.loc[count, '3rd Verse'] = my_lst_str2
    count = count+1

 

In [4]:
data

,Haikuu,1st Verse,2nd Verse,3rd Verse
0,rectory roofers\ntheir ladders\ntake them high...,'rectory roofers,their ladders,take them higher
1,summer cabin\nthe ants\ndo the dishes\n,'summer cabin,the ants,do the dishes
2,lagoon at sunrise?\nthe shadow\nchases its pel...,'lagoon at sunrise?,the shadow,chases its pelican
3,barren trees\neven the tiniest twig\nembraced ...,'barren trees,even the tiniest twig,embraced by the mist
4,windfall apples\nbees tango\nto a waltz\n,'windfall apples,bees tango,to a waltz
...,...,...,...,...
8365,coconut grove\nleaf blades comb\nthe moonlight\n,'coconut grove,leaf blades comb,the moonlight
8366,alpine lake?\nmy?breast'stroke's'shining?arc\n...,"""alpine lake?",my?breast'stroke's'shining?arc,toward sunrise
8367,spruce woods\nfireweed filling\nthe vacancy\n,'spruce woods,fireweed filling,the vacancy
8368,corrugated sun?\nchilies and laundry\nin rooft...,'corrugated sun?,chilies and laundry,in rooftop haze


In [ ]:
haikuu_list=[]
haikuu_string =''

df = data.iloc[:,1:].values
for i in range(len(df)):
    for j in range(0,3):
        haikuu_list.append(df[i][j])
        haikuu_string = ', '.join(map(str, haikuu_list))
haikuu_string

In [ ]:
import spacy
nlp = spacy.load('en',disable=['parser', 'tagger','ner'])

nlp.max_length = 1198623

In [ ]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\'\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

tokens = separate_punc(haikuu_string)
       http://localhost:8888/notebooks/06-Deep-Learning/Untitled.ipynb# 
        

#### Creating Sequences of Tokens

In [ ]:
# organize into sequences of tokens
train_len = 3+1 # 50 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

#### Keras Tokenisation

In [ ]:
from keras.preprocessing.text import Tokenizer
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [ ]:
for i in sequences[]:
    print(f'{i} : {tokenizer.index_word[i]}')

In [ ]:
import numpy as np
sequences = np.array(sequences)
sequences

#### Create Model:LSTM!

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [ ]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 3, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

#### Train / Test Split

In [ ]:
from keras.utils import to_categorical
vocabulary_size = len(tokenizer.word_counts)
X= sequences[:,:-1]
Y = sequences[:,-1]
seq_len = X.shape[1]
Y = to_categorical(Y, num_classes=vocabulary_size+1)
Y


#### Training the Model

In [ ]:
   # define model
model = create_model(vocabulary_size+1, seq_len) 

In [ ]:
from pickle import dump,load
# fit model
model.fit(X, Y, batch_size=128, epochs=300,verbose=1)

## Generating New Text

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
#    INPUTS:
#    model : model that was trained on text data
#    tokenizer : tokenizer that was fit on text data
#    seq_len : length of training sequence
#    seed_text : raw string text to serve as the seed
#    num_gen_words : number of words to be generated by model
    
    # Final Output
    output_text = []
    
    # Intial sentence
    input_text = seed_text
    
    # Create requested number of words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate(making it robust)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Making it look like a sentence.
    return ' '.join(output_text)

#### Grab a random seed sequence

In [ ]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))
random_seed_text = text_sequences[random_pick]
seed_text = ' '.join(random_seed_text)
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=10)